In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl

In [9]:
pip install kaggle


Note: you may need to restart the kernel to use updated packages.


In [10]:
!ls

backpack.ipynb	main.py  playground-series-s5e2.zip


In [11]:
!mkdir -p /teamspace/studios/this_studio/.config/kaggle
!mv kaggle.json /teamspace/studios/this_studio/.config/kaggle/
!chmod 600 /teamspace/studios/this_studio/.config/kaggle/kaggle.json


mv: cannot stat 'kaggle.json': No such file or directory


In [12]:
!kaggle competitions download -c playground-series-s5e2


playground-series-s5e2.zip: Skipping, found more recently modified local copy (use --force to force download)


In [13]:
!unzip playground-series-s5e2.zip -d ./kaggle


Archive:  playground-series-s5e2.zip
  inflating: ./kaggle/sample_submission.csv  
  inflating: ./kaggle/test.csv       
  inflating: ./kaggle/train.csv      


  inflating: ./kaggle/training_extra.csv  


In [14]:
train_df_1= pd.read_csv("kaggle/train.csv")
train_df_1.drop('id',axis=1,inplace = True)
train_df_2=pd.read_csv("kaggle/training_extra.csv")
train_df_2.drop('id',axis=1,inplace=True)
test_df= pd.read_csv("kaggle/test.csv")
test_id = test_df['id']
test_df.drop('id',axis=1,inplace = True)


In [15]:
train_df = pd.concat([train_df_1, train_df_2], axis=0, ignore_index=True)

In [16]:
train_df.shape

(3994318, 10)

In [17]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3994318 entries, 0 to 3994317
Data columns (total 10 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Brand                 object 
 1   Material              object 
 2   Size                  object 
 3   Compartments          float64
 4   Laptop Compartment    object 
 5   Waterproof            object 
 6   Style                 object 
 7   Color                 object 
 8   Weight Capacity (kg)  float64
 9   Price                 float64
dtypes: float64(3), object(7)
memory usage: 304.7+ MB


In [18]:
train_df.isnull().sum()

Brand                   126758
Material                110962
Size                     87785
Compartments                 0
Laptop Compartment       98533
Waterproof               94324
Style                   104180
Color                   133617
Weight Capacity (kg)      1808
Price                        0
dtype: int64

In [19]:
for col in train_df.columns:
    if col == "Price": 
        continue
    if train_df[col].dtype == 'object':
        train_df[col].fillna("Unknown", inplace=True)
        test_df[col].fillna("Unknown", inplace=True)
    else:
        train_df[col].fillna(train_df[col].median(), inplace=True)
        if col in test_df.columns: 
            test_df[col].fillna(test_df[col].median(), inplace=True)

In [20]:
train_df.isnull().sum()

Brand                   0
Material                0
Size                    0
Compartments            0
Laptop Compartment      0
Waterproof              0
Style                   0
Color                   0
Weight Capacity (kg)    0
Price                   0
dtype: int64

In [21]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
categories = ['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof', 'Style', 'Color','Weight Capacity (kg)']

for col in categories:
    train_df[col] = encoder.fit_transform(train_df[col])
    test_df[col] = encoder.fit_transform(test_df[col])

In [22]:
train_df.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,1,1,1,7.0,2,0,2,0,422864,112.15875
1,1,0,2,10.0,2,2,1,3,1695046,68.88056
2,4,1,2,2.0,2,0,1,5,837448,39.17320
3,2,2,2,8.0,2,0,1,3,548130,80.60793
4,0,0,1,1.0,2,2,1,3,926275,86.02312


In [23]:
test_df.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
0,3,1,2,2.0,0,0,2,3,78098
1,2,0,1,7.0,0,2,0,3,39779
2,0,0,0,9.0,0,2,1,1,29525
3,0,2,0,1.0,2,0,1,3,66395
4,5,2,0,2.0,2,2,2,0,20223


In [24]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3994318 entries, 0 to 3994317
Data columns (total 10 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Brand                 int64  
 1   Material              int64  
 2   Size                  int64  
 3   Compartments          float64
 4   Laptop Compartment    int64  
 5   Waterproof            int64  
 6   Style                 int64  
 7   Color                 int64  
 8   Weight Capacity (kg)  int64  
 9   Price                 float64
dtypes: float64(2), int64(8)
memory usage: 304.7 MB


In [25]:
x = train_df.drop('Price', axis=1)
y = train_df['Price']

In [26]:
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [33]:
!pip install xgboost

In [34]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

model = XGBRegressor(
    objective='reg:squarederror',
    random_state=42,
    n_estimators=200,
    learning_rate=0.1,
    max_depth=6,
    enable_categorical=True)

model.fit(x_train, y_train)
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_test, y_pred)
print("Validation Mean Absolute Error (MAE):", mae)

model.save_model('xgboost_model.json')

Validation Mean Absolute Error (MAE): 33.61457388486069


In [35]:
test_predictions = model.predict(test_df)

submission = pd.DataFrame({'id':test_id, 'Price': test_predictions})
submission.to_csv('submission.csv', index=False)
print("Submission file saved as 'submission.csv'.")

Submission file saved as 'submission.csv'.
